In [34]:
from google import genai
from dotenv import load_dotenv
import os
from pydantic import BaseModel, ValidationError
from typing import List
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json
import re

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



In [35]:
def save_json_from_string(input_string, output_filename):
    """
    Extracts JSON content from a string and saves it to a JSON file with the specified name.

    :param input_string: The input string containing JSON data within brackets.
    :param output_filename: The name of the output JSON file (e.g., "data.json").
    :return: True if the operation was successful, False otherwise.
    """
    # Step 1: Extract the JSON content within the brackets
    result = re.search(r'\[.*\]', input_string, re.DOTALL)

    if result:
        extracted_content = result.group(0)  # Get the matched content

        try:
            # Step 2: Parse the extracted content into a Python object (list/dict)
            json_data = json.loads(extracted_content)

            # Step 3: Write the JSON data to a file
            with open(output_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)  # Save with pretty formatting

            print(f"JSON data has been successfully written to '{output_filename}'.")
            return True  # Operation was successful
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except Exception as e:
            print(f"An error occurred while writing the file: {e}")
    else:
        print("No content found within brackets.")

    return False  # Operation failed

In [36]:
MODEL = "gemini-2.0-flash-exp"

In [37]:

COMPANY = 'Proximus'

In [38]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

In [39]:
system_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write an elaborate company report. 

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report."""

In [40]:
config = GenerateContentConfig(system_instruction=system_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)

In [41]:
contents = f"""
Write a concise report about {COMPANY}.

Add to the report a link to the most recent company logo, the company colors in rgb, a summary of companies who offer services to them, a comprehensive summary of the key employees and their roles in the company, a summary of departments of the company. 

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)


Okay, I will create a concise report about Proximus, including the requested information. Here's my plan:

1.  **Company Logo:** Search for the most recent Proximus logo image and provide a link.
2.  **Company Colors:** Search for Proximus's official brand colors and provide their RGB values.
3.  **Service Providers:** Search for companies that provide services to Proximus.
4.  **Key Employees:** Search for key employees at Proximus and their roles.
5.  **Company Departments:** Search for the different departments within Proximus.
6.  **Company Report:** Compile all the information into a concise report.

Now, let's start with the searches.
---
**Proximus Company Report**

**Company Overview:**

Proximus is a leading telecommunications and digital services provider in Belgium, with operations extending into Luxembourg and the Netherlands. It offers a wide array of services, including fixed and mobile telephony, internet, digital television, and IT solutions. The company operates under 

**Proximus Key Employees and Roles**

In [42]:
contents = f"""
Write a report about {COMPANY}

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {{'first_name':str, 'family_name':str, 'role':str, 'department':str}}
Return: list[Proximus_employees]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "first_name": "Guillaume",
        "family_name": "Boutin",
        "role": "Chief Executive Officer",
        "department": "Executive Committee"
    },
    {
        "first_name": "Stefaan",
        "family_name": "De Clerck",
        "role": "Chairman of the Board",
        "department": "Board of Directors"
    },
     {
        "first_name": "Jim",
        "family_name": "Casteele",
        "role": "Chief Consumer Market Officer",
        "department": "Senior Management"
    },
    {
        "first_name": "Antonietta",
        "family_name": "Mastroianni",
        "role": "Chief Digital and IT Officer",
         "department":"Senior Management"
    },
    {
        "first_name": "Mark",
        "family_name": "Reid",
        "role": "Finance Lead",
        "department": "Leadership Squad"
    },
     {
        "first_name": "Ben",
        "family_name": "Appel",
        "role": "Corporate Affairs Lead",
        "department": "Leadership Squad"
    },
    {

In [43]:
save_json_from_string(response.text, 'proximus_employees.json')

JSON data has been successfully written to 'proximus_employees.json'.


True

**Departments Proximus**

In [44]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Proximus_departments = {{'department':str, 'subdivision':list[str]}}
Return: list[Proximus_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "department": "Chief Executive Officer's Department",
        "subdivision": []
    },
    {
        "department": "Consumer Market",
        "subdivision": []
    },
     {
        "department": "Enterprise Market",
        "subdivision": ["Proximus NXT"]
    },
    {
        "department": "Corporate Affairs",
         "subdivision": []
    },
    {
        "department": "Network and Wholesale",
        "subdivision": []
    },
    {
        "department":"Customer Operations",
        "subdivision": []
    },
    {
        "department":"Human Capital",
        "subdivision": []
    },
    {
        "department":"Digital and Information Technology",
        "subdivision": []
    },
    {
        "department":"Finance",
        "subdivision": []
    },
    {
      "department": "Proximus Global",
      "subdivision": ["BICS", "Telesign", "Route Mobile"]
    },
     {
        "department": "Proximus Media House",
        "subdivision": []
    },
    {
        "dep

In [45]:
save_json_from_string(response.text, 'proximus_departments.json')

JSON data has been successfully written to 'proximus_departments.json'.


True

**Company Colors**

In [46]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

Do not include speculative colors and focus only on the main ones.

The report should be written in this style example:

Use this JSON schema:

company_colors = {{'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
Return: list[company_colors]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""


response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "color_name_1": "Primary Purple",
        "hex_code_1": "#5C2D91",
        "rgb_code_1": [92, 45, 145]
    },
    {
        "color_name_2": "White",
        "hex_code_2": "#FFFFFF",
        "rgb_code_2": [255, 255, 255]
    },
    {
        "color_name_3": "Black",
        "hex_code_3": "#000000",
        "rgb_code_3": [0, 0, 0]
    }
]
```


In [47]:
save_json_from_string(response.text, 'proximus_colors.json')

JSON data has been successfully written to 'proximus_colors.json'.


True

**Service providers**

In [48]:
contents = f"""

Write a report about {COMPANY}

List the main service providers of the company. Include any known suppliers, contractors, technology service providers, or any other third-party companies that the company relies on. 
Provide the names of the service providers, type and description of the services they provide to the company.

The report should be written in this style example:

Use this JSON schema:

company_service = {{"provider":str,"service":str,"type":str]}}
Return: list[company_service]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)

print(response.text)

```json
[
    {
        "provider": "Scarlet",
        "service": "Telecommunication services",
        "type": "Subsidiary brand of Proximus, utilizing Proximus infrastructure to provide services."
    },
    {
        "provider": "Mobile Vikings",
        "service": "Telecommunication services",
         "type": "Brand of Proximus, offering mobile services."
    },
    {
        "provider": "Tango",
        "service": "Telecommunication services",
        "type": "Brand of Proximus Luxembourg, offering fixed and mobile telephony, internet and television services."
    },
     {
        "provider": "Telindus Luxembourg",
        "service": "Telecommunication and ICT services",
        "type": "Brand of Proximus Luxembourg, providing telecommunication services, ICT infrastructure, multi-cloud, digital finance solutions, cybersecurity, business applications, managed services and training services."
    },
    {
        "provider": "Telindus Netherlands",
         "service": "Telecommuni

In [49]:
save_json_from_string(response.text, 'proximus_providers.json')

JSON data has been successfully written to 'proximus_providers.json'.


True